In [1]:
import os
from dotenv import load_dotenv

load_dotenv('app/src/shared/.env')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
from langchain_community.document_loaders import PubMedLoader

loader = PubMedLoader("liver", load_max_docs=10)
documents = loader.load()
documents

[Document(page_content='INTRODUCTION: Bronchobiliary fistulas are rare and difficult to treat. Peacock first reported this entity in 1850 while treating a patient with hepatic encopresis.\nCASE PRESENTATION: A 67-year-old Chinese male patient presented to the outpatient clinic with a complaint of coughing up phlegm with chest tightness for 4 days with symptoms of intermittent bilirubin sputum with a sputum volume of about 500\xa0ml per day but no symptoms of abdominal pain or jaundice and no yellow urine or steatorrhea. The examination revealed cyanosis of the lips and mouth, barrel chest, low breath sounds on the right side, and a large number of wet rales heard in both lungs. The imaging investigations were suggestive of bronchobiliary fistula. Therefore, the patient was operated on and discharged with no perioperative complications.\nCONCLUSION: Bronchobiliary fistula should be considered diagnostically in patients with known liver disease who also experience trauma or medical treat

In [5]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# documents = load your documents

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(documents, 10, distributions) 
testset.to_pandas()

embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the role of ASGPR in the selective deg...,[PURPOSE: Immunohistochemical staining of prog...,The ASGPR (asialoglycoprotein receptor) plays ...,simple,"[{'uid': '38910165', 'Title': 'Targeted protei...",True
1,What are the temporal trends in mortality for ...,[BACKGROUNDS/AIMS: The trends in mortality of ...,Annual age-standardized mortality from HCC dec...,simple,"[{'uid': '38910110', 'Title': 'Burden of Morta...",True
2,How can markers of anemia aid in the early det...,[Familial lecithin:cholesterol acyltransferase...,Markers of anemia may aid in the early detecti...,simple,"[{'uid': '38910105', 'Title': 'Longitudinal an...",True
3,How can PET imaging enhance the understanding ...,[PURPOSE: Immunohistochemical staining of prog...,"The LYTAC strategy, enhanced by PET imaging, h...",simple,"[{'uid': '38910165', 'Title': 'Targeted protei...",True
4,What were the symptoms observed in the patient...,[INTRODUCTION: Bronchobiliary fistulas are rar...,The symptoms observed in the patient with bron...,simple,"[{'uid': '38910252', 'Title': 'Bronchobiliary ...",True
5,What factors contribute to higher rates of HCC...,[BACKGROUNDS/AIMS: The trends in mortality of ...,nan,multi_context,"[{'uid': '38910110', 'Title': 'Burden of Morta...",True
6,What are the risks and complications of co-inf...,[We describe the case of a 57-year-old male wi...,nan,multi_context,"[{'uid': '38910238', 'Title': 'Chronic active ...",True
7,What factors contribute to corneal opacity pro...,[Familial lecithin:cholesterol acyltransferase...,Corneal opacity progression in patients with F...,multi_context,"[{'uid': '38910105', 'Title': 'Longitudinal an...",True
8,How can PD-L1 PET imaging and anti-PD-1 therap...,[PURPOSE: Immunohistochemical staining of prog...,PD-L1 PET imaging and anti-PD-1 therapy can en...,multi_context,"[{'uid': '38910165', 'Title': 'Targeted protei...",True
9,What is the relationship between the DNA methy...,"[Fibroblast growth factor (FGF) 21, a hormone ...",The DNA methylation state of the FGF21 gene in...,reasoning,"[{'uid': '38910123', 'Title': 'Analysis of ser...",True


In [6]:
from datasets import Dataset 
from ragas.metrics import faithfulness, answer_correctness
from ragas import evaluate

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[answer_correctness])
score.to_pandas()

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

,question,answer,ground_truth,answer_correctness
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on January 15, 1967",0.749093
1,Who won the most super bowls?,The most super bowls have been won by The New ...,The New England Patriots have won the Super Bo...,0.981086
